In [1]:
from pathlib import Path

from data_scripts import get_newest_data_paths
from mandel_model import make_mandel_setup
from mandel_solvers import make_mandel_solver_space

from solver_selector.simulation_runner import make_simulation_runner

path = Path().parent / "../2"
load_data_paths = get_newest_data_paths(path / "poro_coldstart_s", n_newest=1)
load_data_paths += get_newest_data_paths(path / "poro_coldstart_m", n_newest=1)
assert len(load_data_paths) == 2

solver_space = make_mandel_solver_space(l_factor="dynamic")
simulation = make_mandel_setup(model_size="large")

/home/firedrake/workspace/porepy_workspace/porepy/src/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
simulation_runner = make_simulation_runner(
    solver_space=solver_space,
    params={
        "load_statistics_paths": load_data_paths,
        "print_solver": True,
        "predictor": "gaussian_process",
        # 'regressor': 'mlp',
    },
)

Selecting from 1 solvers.
0 gmres - splitting_fixed_stress [primary - direct, secondary - direct, l_factor=0, primary_variable, method]
Using Gaussian process, exploration: 0.1
Warm start using data:
/home/firedrake/workspace/porepy_workspace/solver_selector/examples/3/../2/performance/poro_coldstart_s_6.npy
/home/firedrake/workspace/porepy_workspace/solver_selector/examples/3/../2/performance/poro_coldstart_m_5.npy


In [3]:
import pandas as pd
import numpy as np

predictor = simulation_runner.solver_selector.predictors[0]
X = np.array(predictor.memory_contexts)
y = np.array(predictor.memory_rewards)

data = pd.DataFrame(X, columns=['ts', 'ms', 'cfl_mean', 'cfl_max', 'l_factor'])
data['target'] = y
data.describe()

,ts,ms,cfl_mean,cfl_max,l_factor,target
count,2.000000e+02,200.000000,200.000000,200.000000,200.000000,200.000000
mean,2.302585e+00,9.168575,4.761302,2.675176,0.622368,0.746518
std,4.452036e-16,0.284955,0.669823,0.074469,0.131955,0.404360
min,2.302585e+00,8.884333,4.250723,2.650625,0.000000,0.116676
25%,2.302585e+00,8.884333,4.380047,2.658663,0.631579,0.366121
50%,2.302585e+00,9.168575,4.566675,2.666951,0.631579,0.642750
75%,2.302585e+00,9.452816,4.896778,2.671508,0.684211,1.168264
max,2.302585e+00,9.452816,9.615875,3.467773,1.000000,1.198279


In [4]:
ts_mean = data.ts.mean()
ms = np.linspace(data.ms.min(), data.ms.max() + (data.ms.max() - data.ms.min()), 100)
cfl_mean = data.cfl_mean.mean()
cfl_max = data.cfl_max.mean()
l_factor = np.linspace(data.l_factor.min(), data.l_factor.max(), 100)

ms, l_factor = np.meshgrid(ms, l_factor, indexing='ij')

data1 = np.zeros((5, 100, 100))
data1[0] = ts_mean
data1[1] = ms
data1[2] = cfl_mean
data1[3] = cfl_max
data1[4] = l_factor
data1 = data1.reshape(5, -1)
data1 = data1.T

target1 = predictor.regressor.predict(data1)
df1 = pd.DataFrame(data1, columns=['ts', 'ms', 'cfl_mean', 'cfl_max', 'l_factor'])
df1['target'] = target1

In [5]:
mean, std = predictor.regressor.predict(data1, return_std=True)
df1['target'] = mean + std

In [6]:
from dash_app import make_app

make_app([data, df1])